Make that we are running the correct version of TensorFlow first

In [2]:
import tensorflow as tf
tf.__version__

'2.0.0-beta1'

In [3]:
import sys

assert sys.version_info >= (3, 6) # Python ≥3.6 required
assert tf.__version__ >= "2.0"    # TensorFlow ≥2.0 required

# Train basic model on the generated emnist-lines dataset

In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sys
# To be able to write code as if this notebook was one level up in files tree structure.
sys.path.append('..')

from pathlib import Path

import tensorflow as tf
from tensorflow.python.ops import array_ops
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras.models import Model as KerasModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import Sequence

import pandas as pd
from PIL import Image

In [5]:
from recognizer.datasets import Dataset
from recognizer.datasets import EmnistDataset
from recognizer.networks import lenet
from recognizer.networks import simple
from recognizer.networks import NetworkInput

In [6]:
model_save_path = Path("../recognizer/weights/lines_all_conv_model.h5")

Hyperparameters

In [7]:
batch_size = 256
epochs = 32

## Load characters mapping

In [8]:
import json

with open(Dataset.raw_data_path()/"emnist"/"emnist_essentials.json") as json_file:  
    mapping = json.load(json_file)["mapping"]
    mapping = {m[0]: m[1] for m in mapping}
    mapping[62] = ' '
    mapping[63] = '_'
    
mapping_reversed = {v: k for k, v in mapping.items()}

## Build the new model

In [33]:
char_image_height = 28
char_image_width = 28
max_length = 34
num_classes = 64
image_height = char_image_height
image_width = char_image_width * max_length

input_shape = (image_height, image_width)
output_shape = (max_length, num_classes)

window_width: float = 16
window_stride: float = 8

In [10]:
def build_model():
    model = Sequential()

    # model.add(layers.Reshape((image_height, image_width, 1), input_shape=input_shape))

    # # Normalize the image
    # model.add(layers.Lambda(lambda x: norm(x, network_input.mean, network_input.std), input_shape=input_shape, output_shape=input_shape))

    model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu' , input_shape=(image_height, image_width, 1)))
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.2))
    # (image_height // 2 - 2, image_width // 2 - 2, 64)

    # -2 because we use the default padding which is 'valid' not 'same'.
    # /2 because we did maxPooling with pool size 2, this halved the output size of the previous conv layer.
    new_height = image_height // 2 - 2
    new_width = image_width // 2 - 2
    # Let us also scale the desired window width and stride accordingly.
    new_window_width = window_width // 2 - 2
    new_window_stride = window_stride // 2
    model.add(layers.Conv2D(128, kernel_size=(new_height, new_window_width), strides=(1, new_window_stride), activation='relu'))
    model.add(layers.Dropout(0.2))
    # (1, num_windows, 128)

    num_windows = int((new_width - new_window_width) / new_window_stride) + 1
    model.add(layers.Reshape((num_windows, 128, 1)))
    # (num_windows, 128, 1)

    width = int(num_windows / max_length)
    model.add(layers.Conv2D(num_classes, kernel_size=(width, 128), strides=(width, 1), activation='softmax'))
    # (image_width / width, 1, num_classes)

    model.add(layers.Lambda(lambda x: tf.squeeze(x, 2)))
    # (max_length, num_classes)

    # Since we floor'd the calculation of width, we might have too many items in the sequence. Take only output_length.
    model.add(layers.Lambda(lambda x: x[:, :max_length, :]))
    
    return model

In [11]:
if model_save_path.exists():
    print("Load saved model")
    
    model = keras.models.load_model(model_save_path)
else:
    print("Build new model")
    
    model = build_model()

Load saved model


W0825 10:41:45.061343 4461979072 deprecation.py:323] From /Users/mostafagazar/anaconda3/envs/py3/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 950, 32)       320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 948, 64)       18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 474, 64)       0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 474, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 118, 128)       589952    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 118, 128)       0         
_________________________________________________________________
reshape (Reshape)            (None, 118, 128, 1)      

## Look at the data

In [13]:
df = pd.read_csv(Dataset.processed_data_path()/"emnist_lines"/"data.csv")

output_length = len(df)

In [14]:
df.head()

,image,sentence
0,0.png,Whats wrong at state _____________
1,1.png,The transducer itself moves the __
2,2.png,There is an ancient and venerable
3,3.png,They destroyed a trading house ___
4,4.png,Alec waited a moment on guard ___


In [15]:
image = Image.open(Dataset.processed_data_path()/"emnist_lines"/"0.png")
image

In [16]:
np.array(image).shape

(28, 952)

## Try it before training

In [17]:
image_numpy = np.array(image).astype(np.float32)
test_batch = image_numpy.reshape(1, image_height, image_width, 1)

preds = model(test_batch)
preds.shape

TensorShape([1, 34, 64])

In [18]:
image

In [19]:
for char_pred in preds[0]:
    print(mapping[np.argmax(char_pred)], end='')

 he s        _  _st   e __________

## Build the generators

Shuffle the data and reset their index

In [24]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,image,sentence
0,8582.png,Now he was just in the late poems
1,2236.png,That boat in the river maybe ____
2,4353.png,They condemned the movie script __
3,1785.png,Some liars logic a wisp of _______
4,2703.png,During the first pass of Phase 3 _


In [25]:
valid_length = int(len(df) * .2)
valid_df = df.iloc[:valid_length]
train_df = df.iloc[valid_length:]

In [26]:
valid_df.tail()

,image,sentence
1995,5215.png,Speeches by the Soviet ambassador
1996,3588.png,They looked up in surprise as ____
1997,7998.png,these are fairly large tough _____
1998,6149.png,You talked about JohnandLinda as _
1999,1404.png,Its nothing to fool with _______


In [27]:
train_df.head()

,image,sentence
2000,4023.png,It is sufficiently small compared
2001,6654.png,They played crack the whip a few _
2002,6890.png,With Lizzie in the barn the ______
2003,6458.png,No they must look the other way __
2004,2868.png,It looked as Gavin had first seen


In [28]:
class LinesDataSequence(Sequence):
    
    def __init__(self, df, batch_size=32):
        self.df = df
        self.batch_size = batch_size

    def __len__(self):
        return len(df) // batch_size

    def __getitem__(self, idx):
        """Return a single batch."""
        begin = idx * self.batch_size
        end = (idx + 1) * self.batch_size

        batch_x = []
        batch_y = []
        for index in range(begin, end):
            row = self.df.iloc[index]
            
            image = Image.open(Dataset.processed_data_path()/'emnist_lines'/row['image'])
            x = np.array(image).astype(np.float32).reshape(image_height, image_width, 1)
            batch_x.append(x)

            y = [mapping_reversed[char] for char in row['sentence']]
            y = to_categorical(y, num_classes).astype(np.int)
            batch_y.append(y)

        return np.array(batch_x), np.array(batch_y)

## Train

Check progress in TensorBoard by running `tensorboard --logdir=logs/lines_time_distributed` in terminal

In [29]:
callbacks = [
    keras.callbacks.TensorBoard(log_dir='../logs/lines_time_distributed')
]

In [30]:
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
history = model.fit_generator(
    LinesDataSequence(train_df, batch_size),
    steps_per_epoch=len(train_df) // batch_size,
    validation_data=LinesDataSequence(valid_df, batch_size),
    validation_steps=len(valid_df) // batch_size,
    epochs=epochs,
    callbacks=callbacks)
history.history.keys()

Epoch 1/32
 2/31 [>.............................] - ETA: 5:32 - loss: 2.9854 - accuracy: 0.2173

KeyboardInterrupt: 

In [ ]:
fig = plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

plt.show()

# Save the model

In [ ]:
model_save_path

In [ ]:
model.save(model_save_path)

## Restore the model

In [ ]:
# model = keras.models.load_model(model_save_path)

## Predict

In [ ]:
model.trainable = False

# Compile the model for trainable changes to take effect.
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
preds = model(test_batch)
preds.shape

In [ ]:
image

In [ ]:
for char_pred in preds[0]:
    print(mapping[np.argmax(char_pred)], end='')